In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
melb = pd.read_csv('../../data/melb_data.csv')

In [3]:
# three approaches
# 1) drop columns with missing values
# 2) imputation
# 3) an extension to imputation

In [5]:
# 1 approach
y = melb.Price
melb_predictors = melb.drop(['Price'], axis=1)

In [7]:
X = melb_predictors.select_dtypes(exclude=['object'])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [10]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

In [12]:
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_val = X_val.drop(cols_with_missing, axis=1)

print('MAE from Approach 1 (Drop columns with missing values):')
print(score_dataset(reduced_X_train, reduced_X_val, y_train, y_val))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


In [13]:
# 2 approach
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

MAE from Approach 2 (Imputation):
178166.46269899711


In [14]:
# 3 approach



# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_val_plus[col + '_was_missing'] = X_val_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val_plus = pd.DataFrame(my_imputer.transform(X_val_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns = X_val_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_val_plus, y_train, y_val))



MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [15]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


In [17]:
df_train = pd.read_csv('../../data/home-data-for-ml-course/train.csv', index_col='Id')

df_test = pd.read_csv('../../data/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
df_train.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = df_train.SalePrice
df_train.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = df_train.select_dtypes(exclude=['object'])
X_test = df_test.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [18]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [19]:
# Function for comparing different approaches
def score_data(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

In [20]:
# Fill in the line below: get names of columns with missing values
miss = [col for col in X_train.columns if X_train[col].isnull().any()] # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(miss, axis=1)
reduced_X_val = X_val.drop(miss, axis=1)

In [21]:
print("MAE (Drop columns with missing values):")
print(score_data(reduced_X_train, reduced_X_val, y_train, y_val))

MAE (Drop columns with missing values):
17837.82570776256


In [22]:
# Fill in the lines below: imputation
imputer = SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(imputer.transform(X_val))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns


In [23]:
print("MAE (Imputation):")
print(score_data(imputed_X_train, imputed_X_val, y_train, y_val))

MAE (Imputation):
18062.894611872147


In [24]:
final_X_train = X_train.drop(miss, axis=1)
final_X_val = X_val.drop(miss, axis=1)

In [25]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_val = model.predict(final_X_val)
print("MAE (Your approach):")
print(mean_absolute_error(y_val, preds_val))

MAE (Your approach):
17837.82570776256


In [26]:
# Fill in the line below: preprocess test data
X_test = X_test.drop(miss, axis=1)
my_imputer = SimpleImputer()
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))


# Imputation removed column names; put them back
imputed_X_test.columns = X_test.columns


final_X_test = imputed_X_test

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [27]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission_2.csv', index=False)